In [37]:
from product_handler import ProductHandler
ph = ProductHandler()
# ph.drop_models()

In [40]:
ph.get_product_by_id('中性上衣_藍_M').

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1207b9600>}

In [36]:
ph.get_detail_data()

,id,detail_ts,quantity,price,type,supplier,note
478,中性上衣_藍_M,2031-05-22 15:40:37.795436,-250.0,600.0,OUT,柏國,None
3202,男短褲_綠_S,2031-05-10 15:40:51.985474,-50.0,2000.0,OUT,柏國,None
288,女壓力褲_綠_S,2031-04-05 15:40:37.120966,450.0,2000.0,IN,柏國,None
900,女上衣_綠_XL,2031-03-29 15:40:39.303621,150.0,2000.0,IN,柏國,None
473,女瑜珈褲_藍_2L,2031-02-24 15:40:37.778104,750.0,600.0,IN,蝦皮,None
...,...,...,...,...,...,...,...
1209,中性排汗衣_綠_XXL,2015-06-13 15:40:40.585786,750.0,1000.0,IN,蝦皮,None
2892,女短褲_紅_L,2015-06-07 15:40:49.231127,750.0,1000.0,IN,柏國,None
1075,男壓力褲_綠_M,2015-05-21 15:40:39.928107,-150.0,2000.0,OUT,蝦皮,None
3549,中性上衣_藍_S,2015-05-04 15:40:54.192235,750.0,600.0,IN,蝦皮,None
